# Métodos directos

## Sección dorada

In [3]:
## module goldSearch
'''
a,b = bracket(f,xStart,h)
Encuentra (a,b) de un punto mínimo de la
función escalar proporcionada por el usuario f(x).
La búsqueda comienza decendiendo desde xStar en pasos de h.

Finds the brackets (a,b) of a minimum point of the
user-supplied scalar function f(x).
The search starts downhill from xStart with a step
length h.
x,fMin = search(f,a,b,tol=1.0e-6)
Método de la sección dorada para determinar x que minimiza
la función escalar proporcionada por el usuario f(x).
El mínimo debe estar entre  (a,b).
'''
import math
def bracket(f,x1,h):
    c = 1.618033989 
    f1 = f(x1)
    x2 = x1 + h 
    f2 = f(x2)
    if f2 > f1:  # Determine la dirección de decenso y cambie el signo de h si es necesario
        h = -h
        x2 = x1 + h; f2 = f(x2)
        if f2 > f1: return x2,x1 - h # Checa si el minumo esta entre x1 - h y  x1 + h
    for i in range (100):    #loop
        h = c*h
        x3 = x2 + h; f3 = f(x3)
        if f3 > f2: 
            return x1,x3
        x1 = x2; x2 = x3
        f1 = f2; f2 = f3
    print("no se encontro ningun minimo")

def search(f,a,b,tol=1.0e-9):
    nIter = int(math.ceil(-2.078087*math.log(tol/abs(b-a))))
    R = 0.618033989
    C = 1.0 - R
    x1 = R*a + C*b; x2 = C*a + R*b #primera extension
    f1 = f(x1); f2 = f(x2)
    for i in range(nIter): #loop principal
        if f1 > f2:
            a = x1
            x1 = x2; f1 = f2
            x2 = C*a + R*b; f2 = f(x2)
        else:
            b = x2
            x2 = x1; f2 = f1
            x1 = R*a + C*b; f1 = f(x1)
    if f1 < f2: return x1,f1
    else: return x2,f2

## Método de Powell

In [6]:
## module powell
''' xMin,nCyc = powell(F,x,h=0.1,tol=1.0e-6)
    Método de Powell para minimizar la función proporcionada por el usuario F(x).
    x    = punto inicial
    h   = inicio de la busqueda utilizado bracket
    xMin = punto minimo
    nCyc = numero de ciclos
'''
import numpy as np
import math

def powell(F,x,h=0.1,tol=1.0e-6):
    
    def f(s): return F(x + s*v)    # F en la dirección v

    n = len(x)                     # Numero de variables 
    df = np.zeros(n)               # Almacenamiento del descenso de F
    u = np.identity(n)             # Almacenamiento de cevtores v
    for j in range(30):            # Numero de ciclos (30)
        xOld = x.copy()            # Salvar punto de inicio
        fOld = F(xOld)
        for i in range(n): #loop de la busqueda de n lineas con la funcion decreciente
            v = u[i]
            a,b = bracket(f,0.0,h)
            s,fMin = search(f,a,b)
            df[i] = fOld - fMin
            fOld = fMin
            x = x + s*v
        v = x - xOld #loop de la busqueda de la ultima linea
        a,b = bracket(f,0.0,h)
        s,fLast = search(f,a,b)
        x = x + s*v
        if math.sqrt(np.dot(x-xOld,x-xOld)/n) < tol: return x,j+1 #convergencia
        iMax = np.argmax(df)  # localizando la direccion de crecimoento para actualizar la direccion de busqueda
        for i in range(iMax,n-1):
            u[i] = u[i+1]
        u[n-1] = v
    print("Powell no converge")    

In [11]:
#EXAMPLE 10.3
from numpy import array
def F(x): return 100.0*(x[1] - x[0]**2)**2 + (1 - x[0])**2
xStart = array([-1.0, 1.0])
xMin,nIter = powell(F,xStart)
print("x =",xMin)
print("F(x) =",F(xMin))
print("Number of cycles =",nIter)

x = [1. 1.]
F(x) = 3.717507015854018e-29
Number of cycles = 12


In [12]:
#EXAMPLE 10.4
from numpy import array
from math import sqrt

def F(x):
    lam = 1.0 # Penalización
    c = x[0]*x[1] - 5.0 # Restricción
    return distSq(x) + lam*c**2 # función de penalizacion

def distSq(x): return (x[0] - 5)**2 + (x[1] - 8)**2
xStart = array([ 1.0,5.0])
x,numIter = powell(F,xStart,0.1)

print("Intersection point =",x)
print("Minimum distance =", sqrt(distSq(x)))
print("xy =", x[0]*x[1])
print("Number of cycles =",numIter)

Intersection point = [0.73306761 7.58776385]
Minimum distance = 4.28679958766998
xy = 5.562343874620907
Number of cycles = 5


## Método de Nelder-Mead (Downhill)

In [14]:
## module downhill
''' x = downhill(F,xStart,side=0.1,tol=1.0e-6)
    Método simple para minimizar el descenso de la función F 
    proporcionada por el usuario, con respecto al vector x.
    xStart = vector x inicial.
    side   = longitud inicial (default= 0.1)
'''
import numpy as np
import math

def downhill(F,xStart,side=0.1,tol=1.0e-6):
    n = len(xStart)                 # Variables
    x = np.zeros((n+1,n)) 
    f = np.zeros(n+1)
  
    x[0] = xStart 
    for i in range(1,n+1):
        x[i] = xStart
        x[i,i-1] = xStart[i-1] + side        
    for i in range(n+1): f[i] = F(x[i]) #valor de F en los vertices
    
    for k in range(500): #loop
        iLo = np.argmin(f) #vertices mayore y menor
        iHi = np.argmax(f)       
        d = (-(n+1)*x[iHi] + np.sum(x,axis=0))/n #calcula el vector d
        if math.sqrt(np.dot(d,d)/n) < tol: return x[iLo] #convergencia
        
      # Reflection
        xNew = x[iHi] + 2.0*d              
        fNew = F(xNew)        
        if fNew <= f[iLo]:        # acepta refleccion
            x[iHi] = xNew
            f[iHi] = fNew
          # Expansion
            xNew = x[iHi] + d               
            fNew = F(xNew)
            if fNew <= f[iLo]:    # acepta expacion
                x[iHi] = xNew
                f[iHi] = fNew
        else:
          # Refleccion
            if fNew <= f[iHi]:    # acepta refleccion
                x[iHi] = xNew
                f[iHi] = fNew
            else:
              # Contraccion
                xNew = x[iHi] + 0.5*d
                fNew = F(xNew)
                if fNew <= f[iHi]: # acepta contraccion
                    x[iHi] = xNew
                    f[iHi] = fNew
                else:
                  for i in range(len(x)):  # usa la contraccion
                        if i != iLo:
                            x[i] = (x[i] - x[iLo])*0.5
                            f[i] = F(x[i])
    print("Demasiadas interaciones en downhill")
    return x[iLo]

In [16]:
## example10_7
import numpy as np
import math
def S(x):
    global perimeter,area
    lam = 10000.0
    perimeter = x[0] + 2.0*x[1]/math.cos(x[2])
    area = (x[0] + x[1]*math.tan(x[2]))*x[1]
    return perimeter + lam*(area - 8.0)**2
xStart = np.array([4.0, 2.0, 0.0])
x = downhill(S,xStart)
area = (x[0] + x[1]*math.tan(x[2]))*x[1]
print("b = ",x[0])
print("h = ",x[1])
print("theta (deg) = ",x[2]*180.0/math.pi)
print("area = ",area)
print("perimetro = ",perimeter)

b =  2.4816069147962976
h =  2.149137386944977
theta (deg) =  30.000018579561125
area =  7.999976717752945
perimetro =  7.444828039523396
